# **Import Modules**

In [1]:
import sys

import os

import random as pyrand

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

sys.path.insert(0, '../input/deeplearning-utils')

In [2]:
from utils.ops import plot

from utils.tf.ops import io, eval as tf_eval

In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os

import matplotlib.pyplot as plt
import matplotlib.cm as cm

import seaborn as sns

from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score

from tqdm import tqdm

In [4]:
from kaggle_secrets import UserSecretsClient
from kaggle_datasets import KaggleDatasets

In [5]:
import tensorflow as tf

from tensorflow.keras import backend, optimizers, regularizers

from tensorflow.keras.layers import *

from tensorflow.keras.models import Sequential, Model

# from tensorflow.keras.applications import *

from tensorflow.keras.layers.experimental import preprocessing

import tensorflow_addons as tfa

# **Config**

In [6]:
def set_seed(tf_seed=0, np_seed=0, py_rand=0, py_hash=0):

    os.environ['PYTHONHASHSEED'] = str(py_hash)
    pyrand.seed(py_rand)

    tf.random.set_seed(tf_seed)
    np.random.seed(np_seed)

In [7]:
class Config:
    
    SEED = 2053
    
    IMAGE_SIZE = (224, 224)
    
    TRAINING = False

In [8]:
set_seed(Config.SEED)

In [9]:
try:
    
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    
    print('Running on TPU ', tpu.master())
    
except ValueError:
  
    tpu = None

if tpu is not None:
    
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)

    strategy = tf.distribute.experimental.TPUStrategy(tpu)

else:
    
    strategy = tf.distribute.get_strategy()


print("REPLICAS: ", strategy.num_replicas_in_sync)

REPLICAS:  1


# **Load Data**

In [10]:
AUTOTUNE = tf.data.experimental.AUTOTUNE

ignore_order = tf.data.Options()
ignore_order.experimental_deterministic = False

In [11]:
src_dir = '../input/ci-sc22-places-and-scene-recognition/'

test_path = tf.io.gfile.glob(src_dir + '/test_images/test_images/*')

print(len(test_path))

3407


In [12]:
def read_test_example(i):
    
    image_name = os.path.split(test_path[i])[1]
    
    image = io.read_image(test_path[i])
    image = tf.expand_dims(image, axis=0)
    image = tf.image.resize(image, Config.IMAGE_SIZE)
    image = tf.cast(image, dtype=tf.float32)

    image /= 255.0
    
    return image_name, image

# **Build Model**

In [13]:
input_shape = (*Config.IMAGE_SIZE, 3)

In [14]:
with strategy.scope():
    
    vgg16 =  tf.keras.models.load_model('../input/cisc22buildvgg16/vgg16.h5', compile=False)
    
    vgg16.load_weights('../input/cisc22buildvgg16/vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5')
    
    outputs = []
    
    outputs.append(vgg16.get_layer('block2_pool').output)
    outputs.append(vgg16.get_layer('block3_pool').output)
    outputs.append(vgg16.get_layer('block4_pool').output)
    outputs.append(vgg16.get_layer('block5_pool').output)
    
    backbone = Model(vgg16.inputs, outputs)

In [15]:
def build_model():
    
    backbone.trainable = False
    
    # ===================================================================================
    
    inputs = Input(shape=input_shape)
    
    inception = backbone(inputs)
    
    z0 = Conv2D(filters=512, kernel_size=(1, 1), activation='linear', kernel_regularizer=None)(inception[0])
    z0 = GlobalAveragePooling2D()(z0)

    z1 = Conv2D(filters=1024, kernel_size=(1, 1), activation='relu', kernel_regularizer=None)(inception[1])
    z1 = Dropout(0.05)(z1)    
    z1 = GlobalAveragePooling2D()(z1)
    
    z2 = Conv2D(filters=1024, kernel_size=(1, 1), activation='relu', kernel_regularizer=None)(inception[2])
    z2 = Dropout(0.1)(z2)    
    z2 = GlobalAveragePooling2D()(z2)
    
    z3 = Conv2D(filters=1024, kernel_size=(1, 1), activation='relu', kernel_regularizer=None)(inception[3])
    z3 = Dropout(0.1)(z3)    
    z3 = GlobalAveragePooling2D()(z3)
    
    z = Concatenate(axis=-1)([z0, z1, z2, z3])
    
    # ===================================================================================

    y_logits = Dense(6, name='y_logits')(z)
    y_proba = Activation('softmax', name='y_proba')(y_logits)
    
    # ===================================================================================

    model = Model(inputs, y_proba)
    
    # ===================================================================================
    
    optimizer = optimizers.Adam(learning_rate=1e-4, beta_1=0.9, beta_2=0.999)
    
    loss = tf.losses.CategoricalCrossentropy(label_smoothing=0.01, name='categorical_crossentropy')
    metrics = tf.metrics.CategoricalAccuracy(name='accuracy_score')
    
    model.compile(optimizer=optimizer, loss=loss, metrics=metrics)
    
    # ===================================================================================

    return model

# **Load Model**

In [16]:
weights_path = tf.io.gfile.glob('../input/cisc22placesandscenevgg16weights/weights-*.h5')

print(len(weights_path))

1


In [17]:
with strategy.scope():

    model = build_model()

model.load_weights(weights_path[0])

In [18]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
model (Functional)              [(None, 56, 56, 128) 14714688    input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 28, 28, 1024) 263168      model[0][1]                      
__________________________________________________________________________________________________
conv2d_2 (Conv2D)               (None, 14, 14, 1024) 525312      model[0][2]                      
____________________________________________________________________________________________

# **Submission**

In [19]:
submission = pd.read_csv(src_dir + 'sample_submission.csv')

submission.drop([0, 1], axis=0, inplace=True)

for i in tqdm(range(len(test_path))):
    
    image_name, image = read_test_example(i)
    
    y = model.predict_on_batch(image).argmax(axis=-1)[0]
    
    submission = submission.append({'image_name': image_name, 'label': y}, ignore_index=True)

100%|██████████| 3407/3407 [01:14<00:00, 45.75it/s]


In [20]:
submission.to_csv('submission.csv', index=False, encoding='utf-8')